In [7]:
import pandas as pd
from pathlib import Path
import os
import zipfile
from datetime import datetime, timedelta

In [8]:
fileName = 'data_for_AsteriosIlektra.zip'
filePath = 'data_for_AsteriosIlektra'
data_dir = Path(filePath)

In [9]:
if not os.path.exists(filePath):
    print('Data have not been extracted.')
    with zipfile.ZipFile(fileName, 'r') as zip_ref:
        zip_ref.extractall()
else:
    print('Data have already been extracted.')

Data have already been extracted.


In [10]:
if not os.path.exists('data.csv'):
    df = pd.concat(
        pd.read_parquet(parquet_file)
        for parquet_file in data_dir.glob('*.parquet')
    )

    df.to_csv('data.csv')
else:
    df = pd.read_csv('data.csv').drop(columns=['Unnamed: 0'])

In [13]:
sub_df = df.iloc[:10000]
sub_df

,start_time_unix,end_time_unix,user_id
0,1482559891,1482559981,8953
1,1486570973,1486570993,3633
2,1481305012,1481305022,3633
3,1528060816,1528060826,3633
4,1474017399,1474017409,3633
...,...,...,...
9995,1529338887,1529338897,8774
9996,1517861512,1517861585,8774
9997,1526452547,1526452678,906
9998,1480942312,1480942413,906


In [14]:
sub_df[['start_time', 'end_time']] = sub_df[['start_time_unix', 'end_time_unix']].apply(pd.to_datetime, unit='s', origin='unix')
sub_df['duration'] = sub_df['end_time'] - sub_df['start_time']

C:\Users\ezara\anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
<ipython-input-14-eaa769ef98c0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['duration'] = sub_df['end_time'] - sub_df['start_time']


In [15]:
def hour_rounder(t):
    if t.minute < 30:
        return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour))
    else:
        return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(minutes=30))

In [16]:
sub_df['range_start_time'] = sub_df['start_time'].apply(lambda x: hour_rounder(x))
sub_df['range_end_time'] = sub_df['range_start_time'].apply(lambda x: x+timedelta(minutes=30))

<ipython-input-16-709eb394d4b1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['range_start_time'] = sub_df['start_time'].apply(lambda x: hour_rounder(x))
<ipython-input-16-709eb394d4b1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['range_end_time'] = sub_df['range_start_time'].apply(lambda x: x+timedelta(minutes=30))


In [17]:
sub_df

,start_time_unix,end_time_unix,user_id,start_time,end_time,duration,range_start_time,range_end_time
0,1482559891,1482559981,8953,2016-12-24 06:11:31,2016-12-24 06:13:01,0 days 00:01:30,2016-12-24 06:00:00,2016-12-24 06:30:00
1,1486570973,1486570993,3633,2017-02-08 16:22:53,2017-02-08 16:23:13,0 days 00:00:20,2017-02-08 16:00:00,2017-02-08 16:30:00
2,1481305012,1481305022,3633,2016-12-09 17:36:52,2016-12-09 17:37:02,0 days 00:00:10,2016-12-09 17:30:00,2016-12-09 18:00:00
3,1528060816,1528060826,3633,2018-06-03 21:20:16,2018-06-03 21:20:26,0 days 00:00:10,2018-06-03 21:00:00,2018-06-03 21:30:00
4,1474017399,1474017409,3633,2016-09-16 09:16:39,2016-09-16 09:16:49,0 days 00:00:10,2016-09-16 09:00:00,2016-09-16 09:30:00
...,...,...,...,...,...,...,...,...
9995,1529338887,1529338897,8774,2018-06-18 16:21:27,2018-06-18 16:21:37,0 days 00:00:10,2018-06-18 16:00:00,2018-06-18 16:30:00
9996,1517861512,1517861585,8774,2018-02-05 20:11:52,2018-02-05 20:13:05,0 days 00:01:13,2018-02-05 20:00:00,2018-02-05 20:30:00
9997,1526452547,1526452678,906,2018-05-16 06:35:47,2018-05-16 06:37:58,0 days 00:02:11,2018-05-16 06:30:00,2018-05-16 07:00:00
9998,1480942312,1480942413,906,2016-12-05 12:51:52,2016-12-05 12:53:33,0 days 00:01:41,2016-12-05 12:30:00,2016-12-05 13:00:00


In [36]:
jumps = sub_df.groupby(['range_start_time', 'range_end_time', 'user_id'])['user_id'].count().reset_index(name ='Num Session')
jumps = sessions.pivot_table('Num Session', ['range_start_time', 'range_end_time'], 'user_id')
jumps = jumps.fillna(0)

In [37]:
jumps.head(5)

,user_id,389,906,1491,1559,1562,2437,3633,5773,6277,6323,6343,6476,8774,8836,8953,9820
range_start_time,range_end_time,,,,,,,,,,,,,,,,
2016-07-02 07:00:00,2016-07-02 07:30:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-07-02 08:30:00,2016-07-02 09:00:00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-07-02 10:00:00,2016-07-02 10:30:00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-07-02 11:30:00,2016-07-02 12:00:00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-07-02 12:00:00,2016-07-02 12:30:00,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
activity = sub_df.groupby(['range_start_time', 'range_end_time', 'user_id'])['duration'].apply(list).reset_index(name='Activity')
activity.head(10)

,range_start_time,range_end_time,user_id,Activity
0,2016-07-02 07:00:00,2016-07-02 07:30:00,3633,[0 days 00:01:57]
1,2016-07-02 08:30:00,2016-07-02 09:00:00,2437,[0 days 00:02:40]
2,2016-07-02 10:00:00,2016-07-02 10:30:00,2437,[0 days 00:00:10]
3,2016-07-02 11:30:00,2016-07-02 12:00:00,2437,[0 days 00:00:10]
4,2016-07-02 12:00:00,2016-07-02 12:30:00,2437,"[0 days 00:00:50, 0 days 00:01:00]"
5,2016-07-02 14:30:00,2016-07-02 15:00:00,9820,[0 days 00:00:50]
6,2016-07-02 16:00:00,2016-07-02 16:30:00,9820,[0 days 00:00:13]
7,2016-07-02 17:00:00,2016-07-02 17:30:00,2437,[0 days 00:28:03]
8,2016-07-02 19:30:00,2016-07-02 20:00:00,2437,[0 days 00:01:00]
9,2016-07-02 19:30:00,2016-07-02 20:00:00,9820,[0 days 00:00:46]
